In [64]:
import pandas as pd

In [65]:
result = pd.read_csv('result_after.csv',encoding='utf-8',sep='|')

In [66]:
result.head()

,id,ret
0,25001,用户来电对01月产生的15.93元的流量费有疑义，我方经系统查询安上网日志向用户解释，用户不...
1,25002,* * * * * * * * 客户对202011-202012月产生的11.12元的流量费...
2,25003,用户来电反映套餐到期没有通知，我方经系统查询告知用户套餐7 折合约到期在2 月底到期，用户强...
3,25004,用户来电反映自己在网厅办理的49元20g的5g升级包，用户称自己没有办理过，已向用户正常解释...
4,25005,用户来电反映自己办理的承诺通信费60元档得12gb 流量与权益（立即生效合约12个月）业务，...


In [67]:
import rouge
def compute_metrics(rg,preds, targets):
    """
    计算rouge指标
    :param rg:实例化的rouge对象
    :param preds: 文本1
    :param targets: 文本2
    :return: 输出为rouge1,rouge2,rougel的加权均值
    """
    score = rg.get_scores(' '.join(preds), ' '.join(targets))
    rouge1 = score[0]['rouge-1']['f']
    rouge2 = score[0]['rouge-2']['f']
    rougel = score[0]['rouge-l']['f']
    # print(round(rouge1*100, 2),round(rouge2*100, 2),round(rougel*100, 2))
    score = round((rouge1 * 0.2 + rouge2 * 0.4 + rougel * 0.4) , 5)
    return score

In [68]:
text1 = "今天天气不错"
text2 = "今天天气不好"
compute_metrics(rouge.Rouge(),text1,text2)

0.8

In [69]:
import re

pattern = r',|;|\?|!|=|，|。|、|；|！|…'

# 这块下面都是在实验
ab=result['ret']
ab0=ab[0]
ab1=ab[1]
result_list01 = re.split(pattern, ab0)
result_list02 = re.split(pattern, ab1)

print(ab[0])
print(result_list01)
print(ab[1])
print(result_list02)

can01=[result_list01[0]]
for i in result_list01:
    judge=0
    if i!="":
        for j in can01:
            if compute_metrics(rouge.Rouge(),i,j)>0.75:
                print(j)
                judge=1
                break
    if judge==0:
        can01.append(i)    
print(can01)

can02=[result_list02[0]]
for i in result_list02:
    judge=0
    if i!="":
        for j in can02:
            if compute_metrics(rouge.Rouge(),i,j)>0.75:
                print(j)
                judge=1
                break
    if judge==0:
        can02.append(i)
print(can02)

compute_metrics(rouge.Rouge(),result_list01[2],result_list01[4])

用户来电对01月产生的15.93元的流量费有疑义，我方经系统查询安上网日志向用户解释，用户不认可，我方按快速处理向用户核减费用，用户不认可要求核实原因，请处理，谢谢。
['用户来电对01月产生的15.93元的流量费有疑义', '我方经系统查询安上网日志向用户解释', '用户不认可', '我方按快速处理向用户核减费用', '用户不认可要求核实原因', '请处理', '谢谢', '']
* * * * * * * * 客户对202011-202012月产生的11.12元的流量费有质疑，自称没有使用，按上网记录解释无效，强烈要求核实处理，请核实处理，谢谢！联系电话：* * * * * * * *
['* * * * * * * * 客户对202011-202012月产生的11.12元的流量费有质疑', '自称没有使用', '按上网记录解释无效', '强烈要求核实处理', '请核实处理', '谢谢', '联系电话：* * * * * * * *']
用户来电对01月产生的15.93元的流量费有疑义
['用户来电对01月产生的15.93元的流量费有疑义', '我方经系统查询安上网日志向用户解释', '用户不认可', '我方按快速处理向用户核减费用', '用户不认可要求核实原因', '请处理', '谢谢', '']
* * * * * * * * 客户对202011-202012月产生的11.12元的流量费有质疑
['* * * * * * * * 客户对202011-202012月产生的11.12元的流量费有质疑', '自称没有使用', '按上网记录解释无效', '强烈要求核实处理', '请核实处理', '谢谢', '联系电话：* * * * * * * *']


0.60357

In [ ]:
for si in range(len(ab)):
    abx = ab[si]
    # print("原句："+abx)
    sen_list = re.split(pattern, abx)
    # print(sen_list)
    canx = [sen_list[0]]
    for i in sen_list:
        judge = 0
        if i!="":
            for j in canx:
                if compute_metrics(rouge.Rouge(),i,j) > 0.75:
                    judge = 1
                    break
            if judge == 0:
                canx.append(i)
    # print(canx)
    res=""
    for s in range(len(canx)):
        if s!=len(canx)-1:
            res=res+canx[s]+"，"
        else:
            res=res+canx[s]+"。"
    # print("去冗："+res)
    ab[si] = res

result['ret'] = ab

In [63]:
result.to_csv('result.csv',  index=False,encoding='utf-8',sep ='|',header =['id','ret'])